In [1]:
from __future__ import division
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime
import numpy as np
from difflib import SequenceMatcher
import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import sys
sys.path.insert(0, '/Users/Naekid/Desktop/capstone-DSI-5/ebay-price-predictor/data-analysis/utilities/')
from plot_learning_curve import plot_learning_curve
from clean_text import clean_text

pd.set_option('display.max_colwidth',100)

## Import Data

**Import cleaned auction data**

In [45]:
auctions = pd.read_pickle('../pickles/auctions.p')

**Clean Title Text**

In [78]:
auctions['title'] = auctions['title'].apply(clean_text)

## Extract Brands,Models from postgres table

In [29]:
dbname='ebay'
user='nathan'
host='localhost'
table_name='category_specifics'

conn = psycopg2.connect("dbname={} user={} host={}".format(dbname, user, host))
cur = conn.cursor()


# Get Brands
SQL = '''SELECT lower("Brand") as "Brands" FROM {table_name} GROUP BY lower("Brand") ORDER BY COUNT(lower("Brand")) DESC;'''.format(table_name=table_name)
brands = pd.read_sql_query(sql=SQL, con=conn)
brands = brands.iloc[:-1] # get rid of "None" row
brands = brands[brands['Brands'].apply(lambda x: len(x.split()) == 1)] # Valid Brand names should only consists of 1 word
brands = brands[brands['Brands'].apply(lambda x: x!='na' and not ('-' in x))] 
brands = brands['Brands'].tolist()

# Get Models
SQL = '''SELECT lower("Model") as "Model" FROM {table_name} GROUP BY lower("Model") ORDER BY COUNT(lower("Model")) DESC;'''.format(table_name=table_name)
models = pd.read_sql_query(sql=SQL, con=conn)
models = models.iloc[:-1] # get rid of "None" row
models = models['Model'].tolist()

# Get Series
SQL = '''SELECT lower("Series") as "Series" FROM {table_name} GROUP BY lower("Series") ORDER BY COUNT(lower("Series")) DESC;'''.format(table_name=table_name)
series = pd.read_sql_query(sql=SQL, con=conn)

Series
0                       nikon d
1                     canon eos
2                    sony alpha
3               canon powershot
4               canon eos rebel
5                 nikon coolpix
6               panasonic lumix
7               sony cyber-shot
8       canon eos digital rebel
9             fujifilm x series
10                      nikon 1
11                     pentax k
12                olympus evolt
13            samsung wb series
14    fujifilm finepix s series
15                  olympus pen
16                   samsung nx
17         canon powershot elph
18                 olympus om-d
19        samsung galaxy camera
20                  ricoh theta
21               olympus stylus
22                    powershot
23                olympus tough
24   fujifilm finepix xp series
25                        lumix
26                  leica d-lux
27                      coolpix
28              kodak easyshare
29                      leica x
..                          ...
167                kodak pixpro
168                    gear 360
169        leica d-lux type 109
170                        hero
171                       d5500
172                           x
173                       tough
174              fujifilm x-e2s
175                    wg-4 gps
176           olympus vr series
177              powershot elph
178                 nikon d7100
179                    pen lite
180                         24x
181             hd3bb0513318b35
182                         a7r
183                        x-t1
184           samsung nx series
185                         70d
186   fujifilm finepix f series
187                           h
188            olympus x-series
189                    nikon d3
190                 quattro dp3
191                       d7100
192                   snaptouch
193                          dx
194                       ricoh
195                   mini-live
196                        None

[197 rows x 1 columns]

## Find Brand
---

In [84]:
def find_brand(title):
    for brand in brands:
        if brand in title:
            return brand

In [85]:
auctions['brand'] = auctions['title'].apply(find_brand)

## Find Model 

### use NLP + Cosine Similarity to find listing Model
---

In [86]:
auctions['model'] = ''

** Delete Lens from Title **

In [124]:
def filter_title(title):
    for brand in brands:
        title = title.replace(brand, '')    
    for w in bad_words:
        title = title.replace(w, '')
    title = re.sub(r"\d+\-\d+","",title)
    title = re.sub(r"\d+\.\d+","",title)
    
    title = re.sub(r"\s\d+\s"," ",title) # deleting numbers
    title = re.sub(r"\s\d+\s"," ",title) # deleting numbers
    title = re.sub(r"\s\-\s"," ",title)  # deleting individual dashes 
    return title

Test

In [125]:
test_title = auctions['title'].iloc[592]
test_title = auctions['title'].iloc[22]
test_title = auctions['title'].iloc[12]
print test_title

bad_words = ['digital','camera','mm','lens','bag','sd','card','new',\
             'used','broken','cracked', 'kit','zoom','power','brand',\
            'package','bag','shutter','body','black','box','original',\
             'battery','charger','slr','mp','accessories']



filter_title(test_title)

olympus e-3 10.1 mp digital slr camera - black body - 23 597 activations


u' e-3    lr    activation'

**Filter titles for bad words**

In [126]:
auctions['filtered_titles'] = auctions['title'].apply(filter_title)

**Train Vectorizer on available Models**

In [148]:
vectorizer = TfidfVectorizer(ngram_range=(1,1),
                            min_df = 1,
                            max_features=5000)

vectorizer = CountVectorizer(ngram_range=(1,1),
                            min_df = 1,
                            max_features=5000)

models_matrix = vectorizer.fit_transform(models)

models_df = pd.DataFrame(models_matrix.todense(), columns=vectorizer.vocabulary_.keys())
print models_df.shape

(1064, 908)


** Transform Titles using Models Vectorizer **

In [165]:
# vectorizer = TfidfVectorizer(ngram_range=(1,2),
#                             min_df = 10,
#                             max_features=5000)

titles_matrix = vectorizer.transform(auctions['filtered_titles'].tolist())
print titles_matrix.shape

(29961, 908)


**Create vectorized titles df**

In [178]:
titles_df = pd.DataFrame(titles_matrix.todense(), columns=vectorizer.vocabulary_.keys())

titles_df = pd.concat(objs=[titles_df,auctions['title'],auctions['filtered_titles'],auctions['brand']], axis=1)

titles_df['similarity_score'] = None
titles_df['model_name'] = ''

titles_df.reset_index(inplace=True) # for using the index column to compare results with the auctions df

(29961, 908)
(29961, 914)


**Use Cosine Similarity to match title with model**

In [207]:
test_df = titles_df.copy()

for test_index in range(20):
    title_vector = test_df.iloc[test_index, 1:-5].values
    
    similarity_matrix = cosine_similarity(X=title_vector.reshape(1,-1), Y=models_matrix)[0]
    
    match_index = np.argmax(similarity_matrix)
    
    test_df['similarity_score'].iloc[test_index] = np.max(similarity_matrix)
    test_df['model_name'].iloc[test_index] = models[match_index]

In [208]:
test_df.ix[:50, ['title','filtered_titles','brand','similarity_score','model_name']]

title  \
0        canon eos rebel sl1 eos 100d 18.0 mp digital slr camera - black kit w stm   
1          sony cyber-shot dsc-rx100 20.2mp digital camera - black brand new cheap   
2       new nib canon eos rebel t6 digital slr camera premium kit 18-55mm 75-300mm   
3             new canon eos rebel t6 dslr bundle 18-55mm 75-300mm lens bag sd card   
4                  sony nex 6 e pz 16-50mm f3.5-5.6 oss power zoom lens kit camera   
5                       brand new lumix fz300 4k 24x f2.8 long zoom digital camera   
6                                  nikon d2h camera package shutter clicks 30 474    
7         panasonic lumix dmc-g7 mirrorless micro four thirds digital camera body    
8                                  sony cyber-shot rx100 ii digital camera - black   
9                                              nikon d3300 18-55 vr ii kit new box   
10          sony cyber-shot dsc-hx300 20.4mp digital camera - black. original box.   
11      as-is canon ds126491 eos rebel t5 digital slr camera kit w battery charger   
12        olympus e-3 10.1 mp digital slr camera - black body - 23 597 activations   
13                                         sony cyber-shot dsc-h300 digital camera   
14            canon eos 7d 18.0 mp digital slr camera w genuine bg-e7 battery grip   
15                                    nikon 1 j3 14.2mp digital camera body black    
16                         sony cyber-shot dsc-tx20 16.2 mp digital camera - green   
17                   pentax k100d super 6.1 mp w da 18-55mm lens 13k shutter count   
18                                 nikon coolpix aw120 digital camera bundle - red   
19                                                   sony dsc-hx300 digital camera   
20      ricoh theta 360 virtual reality vr digital camera - white - like brand new   
21         polaroid snap touch instant print digital camera lcd display brand new    
22       pentax pentax optio wg-3 16.0 mp digital camera - black lots accessories    
23         canon rebel xti 10.1 mp dslr camera silver 18-55mm lens w extra battery   
24                                                   nikon d70 camera 18-55mm lens   
25                          canon powershot sx280 hs 12.1mp digital camera - black   
26   canon powershot sx610 hs full hd wifi digital camera 20.2 mp 18x optical zoom   
27               canon eos 30d 8.2mp digital camera canon lens ef 35-80mm f1 4.5-6   
28                            nikon d3100 dx format 10.2mp digital slr camera body   
29  samsung wb series wb250f 14.2 mp digital smart camera - white touchscreen wifi   
30    nikon d3000 10.2mp digital slr camera - black kit w af-s dx vr 18-55mm lens    
31           canon powershot g10 14.7 mp black point shoot digital camera working    
32                                                                  canon eos body   
33                                                olympus e-pl1 w 14-40 ex reserve   
34                                  panasonic lumix dmc-zs25 16.1mp digital camera   
35                 nikon coolpix l340 20.2 mp digital mint camera - black warranty   
36                  canon eos digital rebel 300d 6.3 mp digital slr camera - black   
37          nikon 1 j1 10.1 mp digital camera - white kit w vr 10-30mm lens 27528    
38                                 canon powershot sx610 hs - wi-fi enabled white    
39                                                                   olympus e-pl1   
40         canon rebel xs dslr digital camera 18-55mm zoom lens ultraviolet filter   
41                          canon eos 50d 15.1 mp digital slr camera - black body    
42             nikon d40x digital slr camera - black 50mm lens sd cards batteries    
43                                                  canon 530 hs digital camera hd   
44                                                    canon eos 40d digital camera   
45                   sony sel 1855 digital camera 18-55mm - great condition - used   
46                           canon 

In [142]:
print auctions.ix[test_index,'title']
print titles_df.ix[test_index, ['title','filtered_titles']]

title_vector = titles_df.iloc[test_index, 1:-5].values

similarity_matrix = cosine_similarity(X=title_vector.reshape(1,-1), Y=models_matrix)[0]

match_index = np.argmax(similarity_matrix)

print np.max(similarity_matrix)

models[match_index]

sony dsc-hx300 digital camera
title              sony dsc-hx300 digital camera
filtered_titles                       dc-hx300  
Name: 19, dtype: object
0.707106781187


'dsc-hx300'

In [20]:
for row_num in range(titles_df.shape[0]):
    if row_num+1 % 100 == 0:
        print 'Extracting model name for item #{} out of {}'.format(row_num, titles_df.shape[0])

    title_vector = titles_df.iloc[row_num, 1:-5].values
    
    similarity_matrix = cosine_similarity(X=title_vector.reshape(1,-1), Y=models_matrix)[0]
    
    match_index = np.argmax(similarity_matrix)
    
    titles_df['similarity_score'] = np.max(similarity_matrix)
    
    titles_df['model_name'].iloc[row_num] = models[match_index]

Extracting model name for item #0 out of 29961


/Users/Naekid/anaconda3/envs/dsi/lib/python2.7/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Extracting model name for item #100 out of 29961
Extracting model name for item #200 out of 29961
Extracting model name for item #300 out of 29961
Extracting model name for item #400 out of 29961
Extracting model name for item #500 out of 29961
Extracting model name for item #600 out of 29961
Extracting model name for item #700 out of 29961
Extracting model name for item #800 out of 29961
Extracting model name for item #900 out of 29961
Extracting model name for item #1000 out of 29961
Extracting model name for item #1100 out of 29961
Extracting model name for item #1200 out of 29961
Extracting model name for item #1300 out of 29961
Extracting model name for item #1400 out of 29961
Extracting model name for item #1500 out of 29961
Extracting model name for item #1600 out of 29961
Extracting model name for item #1700 out of 29961
Extracting model name for item #1800 out of 29961
Extracting model name for item #1900 out of 29961
Extracting model name for item #2000 out of 29961
Extractin

**Results of NLP Model Extraction**

In [30]:
titles_df.ix[0:10,['title','filtered_titles','model_name']]

title  \
0    canon eos rebel sl1 eos 100d 18.0 mp digital slr camera - black kit w stm   
1      sony cyber-shot dsc-rx100 20.2mp digital camera - black brand new cheap   
2   new nib canon eos rebel t6 digital slr camera premium kit 18-55mm 75-300mm   
3         new canon eos rebel t6 dslr bundle 18-55mm 75-300mm lens bag sd card   
4              sony nex 6 e pz 16-50mm f3.5-5.6 oss power zoom lens kit camera   
5                   brand new lumix fz300 4k 24x f2.8 long zoom digital camera   
6                              nikon d2h camera package shutter clicks 30 474    
7     panasonic lumix dmc-g7 mirrorless micro four thirds digital camera body    
8                              sony cyber-shot rx100 ii digital camera - black   
9                                          nikon d3300 18-55 vr ii kit new box   
10      sony cyber-shot dsc-hx300 20.4mp digital camera - black. original box.   

                           filtered_titles            model_name  
0        eo rebel l1 eo 100d    lr    w tm        sl1 / eos 100d  
1           cyber-hot dc-rx100       cheap  cyber-shot dsc-rx100  
2         nib  eo rebel t6  lr  premium                 rebel t6  
3         eo rebel t6 dlr bundle   len  d               rebel t6  
4                nex e pz  f3..6 o   len                  nex-f3  
5                   fz300 4k 24x f long             hc-wx970k 4k  
6                 d2h  packa hutter click                    d2h  
7     dmc-g7 mirrorle micro four third                    dmc-g7  
8                    cyber-hot rx100 ii             dsc-rx100 ii  
9                          d3300  vr ii                    d3300  
10              cyber-hot dc-hx300    .  .             dsc-hx300

## Find Lens Information

** Use Regex to extract lens information **

In [22]:
def find_lens(title):
    lenses = []
    
    lens = re.findall(r"\d+\-\d+m{2}", title)
    if lens:
        lenses.extend(lens)
        return lenses
    
    lens = re.findall(r"\d+m{2}", title)
    if lens:
        lenses.extend(lens)
        return lenses

    lens = re.findall(r"\d+\-\d+", title)
    if lens:
        lens = [l + 'mm' for l in lens]
        lenses.extend(lens)
        return lenses

**Extract**

In [23]:
titles_df['lens'] = titles_df['title'].apply(find_lens)

## Final Result

In [24]:
titles_df.ix[:50,['title','brand','model_name','similarity_score','lens']]

title  \
0        canon eos rebel sl1 eos 100d 18.0 mp digital slr camera - black kit w stm   
1          sony cyber-shot dsc-rx100 20.2mp digital camera - black brand new cheap   
2       new nib canon eos rebel t6 digital slr camera premium kit 18-55mm 75-300mm   
3             new canon eos rebel t6 dslr bundle 18-55mm 75-300mm lens bag sd card   
4                  sony nex 6 e pz 16-50mm f3.5-5.6 oss power zoom lens kit camera   
5                       brand new lumix fz300 4k 24x f2.8 long zoom digital camera   
6                                  nikon d2h camera package shutter clicks 30 474    
7         panasonic lumix dmc-g7 mirrorless micro four thirds digital camera body    
8                                  sony cyber-shot rx100 ii digital camera - black   
9                                              nikon d3300 18-55 vr ii kit new box   
10          sony cyber-shot dsc-hx300 20.4mp digital camera - black. original box.   
11      as-is canon ds126491 eos rebel t5 digital slr camera kit w battery charger   
12        olympus e-3 10.1 mp digital slr camera - black body - 23 597 activations   
13                                         sony cyber-shot dsc-h300 digital camera   
14            canon eos 7d 18.0 mp digital slr camera w genuine bg-e7 battery grip   
15                                    nikon 1 j3 14.2mp digital camera body black    
16                         sony cyber-shot dsc-tx20 16.2 mp digital camera - green   
17                   pentax k100d super 6.1 mp w da 18-55mm lens 13k shutter count   
18                                 nikon coolpix aw120 digital camera bundle - red   
19                                                   sony dsc-hx300 digital camera   
20      ricoh theta 360 virtual reality vr digital camera - white - like brand new   
21         polaroid snap touch instant print digital camera lcd display brand new    
22       pentax pentax optio wg-3 16.0 mp digital camera - black lots accessories    
23         canon rebel xti 10.1 mp dslr camera silver 18-55mm lens w extra battery   
24                                                   nikon d70 camera 18-55mm lens   
25                          canon powershot sx280 hs 12.1mp digital camera - black   
26   canon powershot sx610 hs full hd wifi digital camera 20.2 mp 18x optical zoom   
27               canon eos 30d 8.2mp digital camera canon lens ef 35-80mm f1 4.5-6   
28                            nikon d3100 dx format 10.2mp digital slr camera body   
29  samsung wb series wb250f 14.2 mp digital smart camera - white touchscreen wifi   
30    nikon d3000 10.2mp digital slr camera - black kit w af-s dx vr 18-55mm lens    
31           canon powershot g10 14.7 mp black point shoot digital camera working    
32                                                                  canon eos body   
33                                                olympus e-pl1 w 14-40 ex reserve   
34                                  panasonic lumix dmc-zs25 16.1mp digital camera   
35                 nikon coolpix l340 20.2 mp digital mint camera - black warranty   
36                  canon eos digital rebel 300d 6.3 mp digital slr camera - black   
37          nikon 1 j1 10.1 mp digital camera - white kit w vr 10-30mm lens 27528    
38                                 canon powershot sx610 hs - wi-fi enabled white    
39                                                                   olympus e-pl1   
40         canon rebel xs dslr digital camera 18-55mm zoom lens ultraviolet filter   
41                          canon eos 50d 15.1 mp digital slr camera - black body    
42             nikon d40x digital slr camera - black 50mm lens sd cards batteries    
43                                                  canon 530 hs digital camera hd   
44                                                    canon eos 40d digital camera   
45                   sony sel 1855 digital camera 18-55mm - great condition - used   
46                           canon 

In [25]:
models

['5d mark ii',
 '70d',
 'd40',
 'x100t',
 'd3100',
 'a7',
 'a6300',
 'd3000',
 'dmc-gh4',
 'a6000',
 '6d',
 't3 / eos 1100d',
 'd600',
 'l840',
 'd700',
 'd80',
 'xti / eos 400d',
 '40d',
 'xsi / eos 450d',
 'd90',
 'd60',
 '7d',
 'd70',
 't1i / eos 500d',
 'd7200',
 'd7100',
 't2i / eos 550d',
 'd200',
 '60d',
 'd50',
 'xt / eos 350d',
 't6i / eos d750',
 'j1',
 'd5000',
 'rebel xs',
 'd610',
 't3i / eos 600d',
 'd7000',
 'd3200',
 'rebel t3',
 'l340',
 'rx100 v',
 'd3300',
 'l830',
 '7d mark ii',
 'x-t1',
 'd300',
 'sx410 is',
 'xs / eos 1000d',
 '80d',
 'd5500',
 'x-t10',
 's100',
 'd40x',
 'sx50 hs',
 's110',
 '30d',
 'p510',
 'dmc-lx5',
 'g11',
 'sx40 hs',
 '5d',
 'dsc-h300',
 'sx610 hs',
 '50d',
 'sx530 hs',
 'rebel xs / 1000d',
 'd3',
 'd70s',
 '20d',
 'dsc-rx100',
 'b500',
 'p530',
 'xs / 1000d',
 'rebel xsi / 450d',
 'dsc-rx10 ii',
 'p520',
 't5i / eos 700d',
 'a350',
 'wb350f',
 'a7r',
 'd5100',
 'hero 3+ silver edition',
 'sx510 hs',
 'd5300',
 'a100',
 'l820',
 'a77 ii',
 '

**Test**

In [ ]:
titles_df.ix[:50,['title','filtered_titles']]

Test Cases:
<br>18-55 
<br>28mm
<br>18-55mm 
<br>75-300mm

In [735]:
test_index = 9
test_title = titles_df['title'].iloc[test_index]
print test_title
lenses = []
# lens = re.findall(r"\d+\-\d+m{2}", test_title)
# lenses.extend(lens)
# print lenses
# print re.findall(r"\d+m{2}", test_title)
lenses = find_lens(test_title)
print lenses


nikon d3300 18-55 vr ii kit new box
[u'18-55']


**Development**

In [711]:
test_title = ' 18-55mm '
# test_title = ',28mm '
# test_title = ' 18-55'

print re.findall(r"\d+\-\d+m{2}", test_title)

print re.findall(r"\d+m{2}", test_title)

print re.findall(r"\d+\-\d+", test_title)






# print re.findall(r"\d+\-\d+mm",test_title)

# print re.findall(r"[^\w-]\d+mm",test_title)

# for match in re.findall(r"[^\w]?\d{2,3}\-\d{2,3}[^\w]?",test_title):
#     print match

['18-55mm']
['55mm']
['18-55']
